# BIM relationship detection


Notebook for performing relationship detection using design files.

#### Input:
1. Design file structure extracted from navisworks NWD file.
2. IFC file generated from NWD file
3. graph predictions from GNN

#### Sections:
1. Relationship identification: Identify, extract and (visualize) 
aggregation and connectivity relationships.
2. Visalization: Deprecated
3. IFC to cloud: Create point clouds from each element in IFC file
4. Graph dataset: Create a graph dataset from relationship information
 
 (GNN training available in link_prediction.ipynb notebook)
5. Evaluate GNN: Evaluate predictions from GNN
6. Visalize predictions: Draw IFC element to visualize FP,TP,FNs
7. Analyze dataset and results: Repetition removal, element category analysis


In [ ]:
# setup

%load_ext autoreload
%autoreload 2

In [ ]:
# general
import json
import collections
import math
import uuid
import random
import pickle
import os
from itertools import islice
from pathlib import Path
import numpy as np

# ifc and pointcloud
import ifcopenshell
import open3d as o3d
#import pymeshlab as ml
from compas.geometry import oriented_bounding_box_numpy
from scipy.spatial import distance
from ifcopenshell.util.selector import Selector

# graph 
import dgl
from dgl.data import DGLDataset
import torch

from src.structure import get_systems, get_branches
from src.ifc import draw_relationship, setup_ifc_file
from src.geometry import element_distance
from src.cloud import element_to_cloud
from src.graph import process_nodes, process_edges, IndustrialFacilityDataset, get_node_features, get_edges_from_node_info
from src.icp import run_command
from src.geometry import sq_dist_vect, vector_mag
from src.centerline import get_centerline_deviation, get_centerline_distance
from src.evaluation import *
# vis
# from ipywidgets import interact
# from OCC.Core.Bnd import Bnd_Box, Bnd_OBB
# from OCC.Core.BRepBndLib import brepbndlib_AddOBB
# from OCC.Core.BRepPrimAPI import (BRepPrimAPI_MakeBox, 
# BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder)
# from OCC.Core.gp import gp_Pnt, gp_XYZ, gp_Ax2, gp_Dir

# from utils.JupyterIFCRenderer import JupyterIFCRenderer
from tqdm.notebook import tqdm_notebook as tqdm

In [ ]:
# load design file structure
data_path = "/mnt/c/data/3D_CAD/"
system_dict_file = data_path + "WestDeckBox.nwd_aggregation.json"
#system = 'West-DeckBox-Piping.rvm'
system = 'West-DeckBox-Pipe.rvm'
pipe_path = "pipe_graph/"
sitename = 'east'

blueprint = 'data/sample.ifc'
temp_dir = "output/east2/"
ifcConvert_executable = "scripts/./IfcConvert"

pipe_graph = False
cloi = False
predict_classes = False

if pipe_graph:
    model_path = 'pipe_graph/'
    dist_thresh=0.025
    rough_dist_thresh=0.1
elif cloi:
    model_path = 'cloi/gnn/'
    data_path = 'cloi/'
    dist_thresh=0.02
    rough_dist_thresh=2.0  
else:
#     model_path = 'gnn_params_bbox_only/'
    model_path = 'gnn_params_bmvc/'
    dist_thresh=0.0002
    rough_dist_thresh=0.2

#m = ifcopenshell.open("data/231110AC-11-Smiley-West-04-07-2007.ifc")k

## Relationship Identification

###  aggrgegation relationships

In [ ]:
get_systems(system_dict_file)    

### Topological relationships

In [ ]:
branches = get_branches(system_dict_file)[system]

In [ ]:
# requisties for IFC file creation

create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
m = ifcopenshell.open("../merged.ifc")
owner_history = m.by_type("IfcOwnerHistory")[0]
project = m.by_type("IfcProject")[0]
context = m.by_type("IfcGeometricRepresentationContext")[0]
floor = m.by_type("IfcBuildingStorey")[0]


In [ ]:
# draw links between connected elements, return connections
def visualize_branches(branches, ifc, floor=None, owner_history=None, context=None, draw=False, contiguous=True, dist_thresh=0.002):
    pipe_type = 'IFCPIPESEGMENT'
    fitting_type = 'IFCPIPEFITTING'

    pipe_selector = Selector()
    fitting_selector = Selector()
    pipes = pipe_selector.parse(ifc, '.' + pipe_type)
    fittings = fitting_selector.parse(ifc, '.' + fitting_type)
    fitting_names = [f.Name for f in fittings]
    pipe_names = [p.Name for p in pipes]
    print(pipes[0].Name)

    vis_dict = {}
    for k, val in branches.items():
        vis_elements = []
        connect = True
        for element in val:
            if element in pipe_names:
                vis_elements.append((element, pipe_type, connect))
                connect = True
            elif element in fitting_names:
                vis_elements.append((element, fitting_type, connect))
                connect = True
            else:
                connect = False
        vis_dict[k] = vis_elements

    error_count = 0
    count = 0
    rels = []
    selector = Selector()
    
    # enumerate through branches
    for k, val in tqdm(vis_dict.items()):
#         if count == 10:
#             break
        branch_size = len(val)
        for i, element in enumerate(val):
            #check if element is not the last element
            if (i+1) < branch_size:
                try:
                    
                    if val[i+1][2] or not contiguous:
                        rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                        if draw:
                            element1 = selector.parse(
                                m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                            element2 = selector.parse(
                                m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                            draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc, floor, owner_history, context)
                
                    else:
                        element1 = selector.parse(
                            m, '.' + element[1] + '[Name *= "' + element[0] + '"]')[0]
                        element2 = selector.parse(
                            m, '.' + val[i+1][1] + '[Name *= "' + val[i+1][0] + '"]')[0]
                        
                        if element_distance(element1, element2, ifc) < dist_thresh:
                            rels.append([(element[0], element[1]), 
                                  (val[i+1][0], val[i+1][1])])
                            if draw:
                                draw_relationship(element[0], element1, 
                                  val[i+1][0], element2, ifc, floor, owner_history, context)
                except Exception as e:
                    #print (e)
                    error_count +=1
        count +=1

    print(error_count)
    return rels


In [ ]:
rels = visualize_branches(branches, m, floor, owner_history, context, True)
# rels = visualize_branches(branches, m)

In [ ]:
print(len(rels), rels[0])

In [ ]:
import pickle
with open('../top_rels_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(rels, f)

In [ ]:
m.write('../east_vis_test.ifc')

## Visualization

In [ ]:
# viewer = JupyterIFCRenderer(m, size=(400,300))
# viewer.setAllTransparent()
# viewer

### aggregation relationships

In [ ]:
# picker = viewer.colorPicker()
# picker


In [ ]:
# out_dict = {'HVAC':['Rohrtypen:Kupfer - Hartgelötet:7718868', 'Rohrtypen:Kupfer - Hartgelötet:7718886', ],
#            'electrical':[ 'Rohrtypen:Kupfer - Hartgelötet:7718872', 'Rohrtypen:Kupfer - Hartgelötet:7718880']}

# # PAINT A SET OF ELEMENTS IN ONE COLOUR
# def systemSelect(system):
#     selector = Selector()
#     for e in out_dict[system]:
#         element = selector.parse(m, '.IfcProduct[Name *= "' + e + '"]')[0]
#         viewer.setColor(element, picker.value)
#     return system

# interact(systemSelect, system=['HVAC', 'electrical'])

Instances of building elements with represenations can be selected interactivly. Information such as the attributes `GUID`, `Name` etc. are displayed to the left of the 3D viewport.

In [ ]:
# # reset colours
# viewer.setDefaultColors()

### topological relationships



#### replace jupyter renderer

1. Compute the bounding box of ifc product directly from points
2. generate ifc elements to indicate relationships


In [ ]:
# test

# element_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element_type = "IFCPIPESEGMENT"

# selector = Selector()
# element = selector.parse(
#     m, '.' + element_type + '[Name *= "' + element_name + '"]')[0]

# shape = element.Representation.Representations[0].Items[0]
# element_coords = np.array(shape.Coordinates.CoordList)
# #print(element_coords)
# bbox = oriented_bounding_box_numpy(element_coords)
# print(bbox)

In [ ]:
# test

# element1_name = "TUBE 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element1_type = "IFCPIPESEGMENT"
# element2_name = "ELBOW 1 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element2_type = "IFCPIPEFITTING"

# element3_name = "TUBE 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element3_type = "IFCPIPESEGMENT"
# element4_name = "ELBOW 2 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element4_type = "IFCPIPEFITTING"
# element5_name = "TUBE 3 of BRANCH /AM-8120227-WD-MDA-01/B1"
# element5_type = "IFCPIPESEGMENT"

# draw_relationship(element1_name, element1_type, 
#                   element2_name, element2_type, m)
# draw_relationship(element2_name, element2_type, 
#                   element3_name, element3_type, m)
# draw_relationship(element3_name, element3_type, 
#                   element4_name, element4_type, m)
# draw_relationship(element4_name, element4_type, 
#                   element5_name, element5_type, m)
#element1_center, element1_coords = get_element_deets()



# centerpoint =gp_Pnt(element1_center)
# ball = BRepPrimAPI_MakeSphere(centerpoint, 0.02).Shape()

In [ ]:
# element.Representation.Representations[0].Items[0].CoordIndex = element.Representation.Representations[0].Items[0].CoordIndex[:1]

## IFC to cloud

sample points from ifc model

In [ ]:
# parse through individual ifc files for each model type and sample point clouds
#ifc = ifcopenshell.open(data_path +"deckboxelbow_ref.ifc")
ifc = ifcopenshell.open(temp_dir +"TEE.ifc")


In [ ]:
element_type = 'IFCPIPEFITTING'
#element_type = 'IFCPIPESEGMENT'
selector = Selector()
tees = selector.parse(ifc, '.' + element_type)
print(len(tees))

In [ ]:
# normalise an element and convert it into an obj
def element_to_obj(element, save_path, blueprint, temp_dir, ifcConvert_executable):
    # setup new ifc
    ifc = setup_ifc_file(blueprint)
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]

    ifc_info = {"owner_history": owner_history,
        "project": project,
       "context": context, 
       "floor": floor}
    
    # normalise points
    points = np.array([e for e in ifc.traverse(element) if e.is_a("IfcCartesianPointList3D")][0][0])
    mean = np.mean(points, axis=0)
    norm_points = points - mean
    norm_factor = np.max(np.linalg.norm(norm_points, axis=1))*3/1000
    norm_points /= norm_factor
    #print(norm_factor, mean, element.id())
    [e for e in ifc.traverse(element) if e.is_a("IfcCartesianPointList3D")][0][0] = norm_points.tolist()

    # add element to new ifc
    ifc.add(element)
    tmp_ifc = os.path.join(temp_dir, 'tmp.ifc')
    ifc.write(tmp_ifc)
    
    # convert ifc to obj
    cmds = (ifcConvert_executable, tmp_ifc, save_path)
    result = run_command(cmds)
    
    return mean.tolist(), norm_factor
    

In [ ]:
# convert an element to an obj
metadata_path = data_path + "bp_east_metadata.json"
category = {}
with open(metadata_path, "r") as jsonFile:
    metadata = json.load(jsonFile)

for i, element in enumerate(tqdm(tees)):
    save_path = data_path +"east_tee_obj/" + str(i) + ".obj"
    mean, norm_factor = element_to_obj(element, save_path, blueprint, temp_dir, ifcConvert_executable)
    category[str(i)] = {"mean":mean, "norm_factor":norm_factor, "id":element.id()}
    
metadata["tee"] = category
with open(metadata_path, "w") as jsonFile:
    json.dump(metadata, jsonFile)

In [ ]:
# convert an element to a point cloud directly
metadata_path = data_path + "bp_east_metadata.json"
category = {}
with open(metadata_path, "r") as jsonFile:
    metadata = json.load(jsonFile)

for i, element in enumerate(tqdm(tees)):
    save_path = data_path +"east_tee_cloud/" + str(i) + ".pcd"
    cloud = element_to_cloud(element, save_path, 2048)
    category[str(i)] = {"id":element.id()}
    
#print(category)
metadata["tee"] = category
with open(metadata_path, "w") as jsonFile:
    json.dump(metadata, jsonFile)

## Graph dataset

### get nodes & edges

In [ ]:
types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']
node_info = process_nodes(ifc, types)

In [ ]:
print(len(node_info), len(node_info[0]), len(node_info[1]))

with open('../nodes_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(node_info, f)

In [ ]:
with open('../top_rels_eastdeckbox.pkl', 'rb') as f:
    rels = pickle.load(f)
with open('../nodes_eastdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    
edges = process_edges(ifc, nodes, rels)

In [ ]:
with open('../edges_eastdeckbox_test.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [ ]:
# sanity checks

# print(points.shape, labels.shape, centers.shape, lengths.shape, directions.shape)
# print(points[0][0], labels[0], centers[0], lengths[0], directions[0])
# print(5 in labels)

# edges_src = edges[:,0]
# edges_dst = edges[:,1]
# print(edges_src)
# print(np.max(edges_dst))

### create dataset

In [ ]:
dataset = IndustrialFacilityDataset()
graph = dataset[0]

print(graph)

## Evaluate GNN

In [ ]:
# load edges

with open(model_path + '/eval/pos_edges_test.pkl', 'rb') as f:
            u, v = pickle.load(f)
with open(model_path + '/eval/neg_edges_test.pkl', 'rb') as f:
            neg_u_full,neg_v_full = pickle.load(f)
print(len(neg_u_full), len(u))

u = u.cpu().numpy()
v = v.cpu().numpy()


In [ ]:
# load predicted scores
with open(model_path + '/eval/pos_score_test.pkl', 'rb') as f:
            pos_score = pickle.load(f)
with open(model_path + '/eval/neg_score_test.pkl', 'rb') as f:
            neg_score= pickle.load(f)


Calculate confusion matrix coefficiants

In [ ]:
threshold = 0.01

# TPs / FNs
sig = 1/(1 + np.exp(-pos_score))
ind = np.arange(len(pos_score))
tp = ind[sig > threshold]
TPs = [(u[i], v[i]) for i in tp]
fn = ind[sig < threshold]
FNs = [(u[i], v[i]) for i in fn]

# FPs
sig = 1/(1 + np.exp(-neg_score))
ind = np.arange(len(neg_score))
fp = ind[sig > threshold]
FPs = [(neg_u_full[i], neg_v_full[i]) for i in fp]
#FPs = fp
print(len(TPs), len(FNs), len(FPs), FPs[20])


In [ ]:
#print(1/(1 + np.exp(-neg_score[0])))

In [ ]:
# with open(model_path + '/eval/metrics_test.pkl', 'wb') as f:
#     pickle.dump([TPs, FPs, FNs], f)

In [ ]:
# with open(model_path + '/eval/metrics_test.pkl', 'rb') as f:
#     TPs, FPs, FNs = pickle.load(f)

Calculate raw metrics

In [ ]:
precision = len(TPs)/(len(TPs)+len(FPs))
recall = len(TPs)/(len(TPs)+len(FNs))
accuracy = (len(TPs)-len(FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

Refinement based on element distances (eliminate predictions beyond a distance threshold)

In [ ]:
if cloi:
    site = 'cloi'
else:
    site = 'eastdeckbox'

node_file = "nodes_" + site + ".pkl"
with open(data_path + node_file, 'rb') as f:
    node_info = pickle.load(f)

In [ ]:
if pipe_graph:
    # filter out non-pipe nodes and recode edges with new edge indices
    node_info0 = [ni for i, ni in enumerate(node_info[0]) if ni[0]==3]
    node_info1 = [node_info[1][i] for i, ni in enumerate(node_info[0]) if ni[0]==3]
    node_info = [node_info0, (node_info1)]
    
    print(len(node_info0), len(node_info1),  len(TPs))

In [ ]:
# refined_TPs = check_predictions(TPs, node_info[1])
# refined_FPs = check_predictions(FPs, node_info[1])

refined_TPs, new_FNs = check_predictions_fast(TPs, node_info[1], node_info[0], 
                                              dist_thresh, rough_dist_thresh)
refined_FPs, _ = check_predictions_fast(FPs, node_info[1], node_info[0],
                                        dist_thresh, rough_dist_thresh)

print(len(refined_TPs), len(refined_FPs), len(new_FNs))

In [ ]:
with open(model_path + '/eval/refined_test.pkl', 'wb') as f:
    pickle.dump([refined_TPs, refined_FPs, new_FNs], f)

In [ ]:
with open(model_path + '/eval/refined_test.pkl', 'rb') as f:
    refined_TPs, refined_FPs, new_FNs = pickle.load(f)
print(len(refined_TPs))

In [ ]:
FNs = FNs + new_FNs
precision = len(refined_TPs)/(len(refined_TPs)+len(refined_FPs))
recall = len(refined_TPs)/(len(refined_TPs)+len(FNs))
accuracy = (len(refined_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
f1 = (2 * precision * recall) / (precision + recall)
print("pr", precision, "re", recall, "ac", accuracy, "f1", f1)

In [ ]:
f1 = (2 * 83.6 * 85.4) / (83.6 + 85.4)
print(f1)

In [ ]:
radius_threshold = 0.1
centerline_angle_threshold = math.radians(90) # angle threshold for two pipes to be considered parallel
centerline_dist_threshold = 0.001 # centerline distance threshold pipes to be to considered connected
params_path = Path('output/east_ref/')
dataset = "east"

refined_refined_TPs, new_new_FNs = connectivity_refinements(refined_TPs, node_info, params_path, dataset, radius_threshold,
                                              centerline_angle_threshold, centerline_dist_threshold)

refined_refined_FPs, _ = connectivity_refinements(refined_FPs, node_info, params_path, dataset, radius_threshold,
                                              centerline_angle_threshold, centerline_dist_threshold)


In [ ]:
print(len(refined_refined_TPs)/len(refined_TPs), len(refined_refined_FPs)/len(refined_FPs))

Refined metrics

In [ ]:
precision = len(refined_refined_TPs)/(len(refined_refined_TPs)+len(refined_refined_FPs))
recall = len(refined_refined_TPs)/(len(refined_refined_TPs)+len(FNs)+len(new_new_FNs))
accuracy = (len(refined_refined_TPs)-len(refined_refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

ROC curve

In [ ]:
pos_score= pos_score[:int(len(pos_score)/10)]
neg_score= pos_score[:int(len(neg_score)/10)]

In [ ]:
len(neg_score)

In [ ]:
scores = np.concatenate([pos_score, neg_score])

In [ ]:
sig = 1/(1 + np.exp(-scores))

In [ ]:
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt


RocCurveDisplay.from_predictions(labels, scores)
plt.show()

### class predictions

## visualize predictions

In [ ]:
ifc = ifcopenshell.open(data_path+"/east_merged.ifc")

In [ ]:
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)
owner_history = ifc.by_type("IfcOwnerHistory")[0]
project = ifc.by_type("IfcProject")[0]
context = ifc.by_type("IfcGeometricRepresentationContext")[0]
floor = ifc.by_type("IfcBuildingStorey")[0]


In [ ]:
red = ifc.createIfcColourRgb('red', Red=0.9, Green=0.0, Blue=0.0)
green = ifc.createIfcColourRgb('green', Red=0.0, Green=0.9, Blue=0.0)
yellow = ifc.createIfcColourRgb('yellow', Red=0.9, Green=0.9, Blue=0.0)

In [ ]:
print(len(refined_TPs), len(refined_FPs), len(FNs))

In [ ]:
# visualize results on ifc file
def draw_predictions(preds, nodes, ifc, colour):
    for pair in tqdm(preds):
        element1 = ifc.by_id(nodes[pair[0]][4])
        element1_name = element1.Name
        element2 = ifc.by_id(nodes[pair[1]][4])
        element2_name = element2.Name
        
        draw_relationship(element1_name, element1, element2_name, 
                          element2, ifc, floor, owner_history, context, colour)

In [ ]:
# visualize pipe dataset
if pipe_graph:
    with open(pipe_path + 'pipe_edges2_' + sitename + '.pkl', 'rb') as f:
        pipe_edges = pickle.load(f)
    pipe_edges = [e[0] for e in pipe_edges]
    print(len(pipe_edges))

In [ ]:
if pipe_graph:
    draw_predictions(pipe_edges, node_info[0], ifc, green)

In [ ]:
draw_predictions(refined_TPs, node_info[0], ifc, green)

In [ ]:
draw_predictions(refined_FPs, node_info[0], ifc, yellow)

In [ ]:
draw_predictions(FNs, node_info[0], ifc, red)

In [ ]:
ifc.write(model_path + '/pipe2_vis_test_ref.ifc')

## Additional refinements

### remove repetitions

In [ ]:
print(len(refined_TPs))
non_rep_TPs = remove_repetitions(refined_TPs)
print(len(non_rep_TPs))

In [ ]:
print(len(refined_FPs))
non_rep_FPs = remove_repetitions(refined_FPs)
print(len(non_rep_FPs))

In [ ]:
print(len(FNs))
non_rep_FNs = remove_repetitions(FNs)
print(len(non_rep_FNs))

In [ ]:
refined_FNs = compare_preds(non_rep_FNs, non_rep_TPs)
print(len(non_rep_FNs), len(refined_FNs))

In [ ]:
print(non_rep_FNs[0])

Refined metrics after removing repetitions

In [ ]:
precision = len(non_rep_TPs)/(len(non_rep_TPs)+len(non_rep_FPs))
recall = len(non_rep_TPs)/(len(non_rep_TPs)+len(non_rep_FNs))
accuracy = (len(non_rep_TPs)-len(refined_FPs) +len(neg_score))/(len(neg_score)+len(pos_score))
print(precision, recall, accuracy)

In [ ]:
with open(model_path + '/eval/non_rep_test.pkl', 'wb') as f:
    pickle.dump([non_rep_TPs, non_rep_FPs, non_rep_FNs], f)

Calculate metrics for each element category

### Analyse dataset and results

In [ ]:
tp_bins = sort_type(non_rep_TPs,node_info[0] )
fp_bins = sort_type(non_rep_FPs,node_info[0] )
fn_bins = sort_type(non_rep_FNs,node_info[0] )

In [ ]:
print(tp_bins)

In [ ]:
recall = tp_bins/(tp_bins+fn_bins)
precision = tp_bins/(tp_bins+fp_bins)

In [ ]:
print(precision)
print(recall)

In [ ]:
print("F1:", (2*precision*recall)/(precision+recall))

Calculate element types in dataset

In [ ]:
# analyse dataset

# load data
site = 'east'

c_east = analyse_dataset('east')
c_west = analyse_dataset('west')
count = c_east + c_west
print(count)

### Dataset creation v2

The dataset must have shared element ids across both the pointclouds used for parameter inference and the graph node dataset. The steps for creating the new dataset are;
1. create subsets of merged.ifc for the 4 classes (include bend in elbow) preserving element id
2. run refinement to get refined_ifcs (manual deletion)
3. generate pointcloud dataset from ifcs
4. get the element ids of refined ifcs, compare with ids in the graph and delete extra nodes / edges to get refined graph

1. create subsets of merged.ifc for the 4 classes (include bend in elbow) preserving element id

In [ ]:
def get_elements_from_ifc(ifc):
    ifc_full = ifcopenshell.open(ifc)
    selector = Selector()
    element_type = 'IFCPIPEFITTING'
    elements_fitting = selector.parse(ifc_full, '.' + element_type)
    element_type = 'IFCPIPESEGMENT'
    elements_segments = selector.parse(ifc_full, '.' + element_type)
    elements = elements_segments + elements_fitting
    print(len(elements))
    return elements

In [ ]:
# load ifc
elements = get_elements_from_ifc(data_path +"east_merged.ifc")

In [ ]:
# load nodes
with open(data_path + 'nodes_eastdeckbox.pkl', 'rb') as f:
    node_info = pickle.load(f)
    nodes = node_info[0]
    
print(len(nodes))

# load edges
with open(data_path + 'edges_eastdeckbox.pkl', 'rb') as f:
    edges = pickle.load(f)
    
print(edges[0])

In [ ]:
# get matching elements between nodes and elements
matching_elements = {}

for i, el in enumerate(tqdm(elements)):
    for j, node in enumerate(nodes):
        if el.id() == node[4]:
            matching_elements[str(j)] = i
            

In [ ]:
b = [b for b in nodes if b[0]]
print(len(matching_elements), nodes[0])

In [ ]:
# filter by class
#types = ['FLANGE', 'ELBOW', 'TEE', 'TUBE', 'BEND']
types = ['BEND']
combined_metadata = {}
for k, tp in enumerate(types):
    class_metadata = {}
    
    # setup new ifc
    ifc = setup_ifc_file(blueprint)
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]
    k = 4
    # add element to new ifc
    for i, nd in enumerate(tqdm(nodes)):
        # check if class matches
        #print(matching_elements[str(i)])
        count = 0
        if nd[0] == k:
            el = elements[matching_elements[str(i)]]
            new_id = ifc.add(el).id()
            class_metadata[str(new_id)] = el.id()
            #print(el.id(), new_id)
            
    combined_metadata[tp] = class_metadata
    
    # write ifc
    tmp_ifc = os.path.join(temp_dir, tp+'.ifc')
    ifc.write(tmp_ifc)
    


In [ ]:
print(len(combined_metadata["BEND"].keys()))

In [ ]:
f = open(os.path.join(temp_dir, 'id_metadata.json'), 'w')
json.dump(combined_metadata, f)

step 2 and 3 must be carried out using IFCPARSE script (or manually) and using element_to_cloud function (or using blender for tees)

4. get the element ids of refined ifcs, compare with ids in the graph and delete extra nodes / edges to get refined graph

In [ ]:
# identify nodes which have been deleted
total_ids = []
for k, tp in enumerate(types):
    elements = get_elements_from_ifc(os.path.join(temp_dir, tp+'_ref.ifc'))
    element_ids = [combined_metadata[tp][str(el.id())] for el in elements]
    total_ids += element_ids

print(len(total_ids))
missing_count = 0

for nd in nodes:
    if nd[4] not in total_ids:
        missing_count += 1
print("missing", missing_count)


In [ ]:
print(len(set(total_ids)))

In [ ]:
# create modified graph with refined nodes
new_nodes, new_node_points, new_edges = [], [], []
id_dict = {}
for i in range(len(nodes)):
    id_dict[str(i)] = None
    
for i, nd in enumerate(nodes):
    if nd[4] in total_ids:
        new_nodes.append(nd)
        new_node_points.append(node_info[1][i])
        id_dict[str(i)] = len(new_nodes)-1
        
print(len(new_nodes), new_nodes[0])

for e in edges:
    if id_dict[str(e[0])] is not None and id_dict[str(e[1])] is not None:
        new_edges.append((id_dict[str(e[0])], id_dict[str(e[1])]))
        
print(len(new_edges), new_edges[0])

In [ ]:
with open(os.path.join(temp_dir, 'nodes_westdeckbox_ref.pkl'), 'wb') as f:
    pickle.dump([new_nodes, new_node_points], f)
    
with open(os.path.join(temp_dir, 'edges_westdeckbox_ref.pkl'), 'wb') as f:
    pickle.dump(new_edges, f)
